### MI using SKLearn

## TODO

- how much does the sklearn differ from the truth with diff nos of bins and diff no of samples? The results completely differ!
- if the vals differe does the relative order change? There is no relation between the relative order between histogram approximation and RBIG.
- does it impact the conclusion eg. XA > XY in sklearn does RBIG gives out similar result? NO


In [29]:
import pandas as pd
import itertools
import os
from datetime import datetime
import pandas as pd
from sklearn.metrics import mutual_info_score
from sklearn.preprocessing import OrdinalEncoder
from tqdm import tqdm 
import itertools as itert
import numpy as np
from batch_run_task_fairness import *

In [30]:
adult_dataset = pd.read_csv('adult_dataset.csv')
adult_dataset.drop('Unnamed: 0', axis = 1, inplace = True)

In [31]:
adult_dataset.columns


Index(['age', 'education-num', 'sex', 'capital-gain', 'capital-loss',
       'hours-per-week', 'income-per-year', 'education', 'marital-status',
       'native-country', 'occupation', 'race', 'relationship', 'workclass'],
      dtype='object')

In [32]:
features_num = ['hours-per-week', 'capital-gain', 'capital-loss'] # removed education-num
features_cat =  ['workclass','education', 'relationship', 'occupation'] #removed native-country
protected_attributes = ['sex','race','age', 'marital-status']
target = 'income-per-year'

In [33]:
adult_dataset[protected_attributes].nunique()

sex                2
race               5
age               74
marital-status     7
dtype: int64

In [34]:
max(adult_dataset['age'])

90.0

In [35]:
min(adult_dataset['age'])

17.0

In [37]:
adult_dataset['age_bins'] = pd.cut(adult_dataset['age'], bins=[1, 20, 40, 60, 80, 100],
                    labels=['1 to 20', '21 to 40', '41 to 60',
                            '61 to 80', '81 to 100'])
adult_dataset.head()

,age,education-num,sex,capital-gain,capital-loss,hours-per-week,income-per-year,education,marital-status,native-country,occupation,race,relationship,workclass,age_bins
0,25.0,7.0,1.0,0.0,0.0,40.0,0.0,11th,Never-married,United-States,Machine-op-inspct,Black,Own-child,Private,21 to 40
1,38.0,9.0,1.0,0.0,0.0,50.0,0.0,HS-grad,Married-civ-spouse,United-States,Farming-fishing,White,Husband,Private,21 to 40
2,28.0,12.0,1.0,0.0,0.0,40.0,1.0,Assoc-acdm,Married-civ-spouse,United-States,Protective-serv,White,Husband,Local-gov,21 to 40
3,44.0,10.0,1.0,7688.0,0.0,40.0,1.0,Some-college,Married-civ-spouse,United-States,Machine-op-inspct,Black,Husband,Private,41 to 60
4,34.0,6.0,1.0,0.0,0.0,30.0,0.0,10th,Never-married,United-States,Other-service,White,Not-in-family,Private,21 to 40


In [38]:
cat = ['workclass','education', 'relationship', 'occupation']#removed native-country
num = ['hours-per-week','capital-gain','capital-loss']# removed education-num
protected_attributes =  ['sex','race','age_bins', 'marital-status']
target = 'income-per-year'

In [39]:
adult_dataset[protected_attributes].nunique()

sex               2
race              5
age_bins          5
marital-status    7
dtype: int64

In [40]:
adult_dataset[cat].nunique()

workclass        7
education       16
relationship     6
occupation      14
dtype: int64

In [41]:
adult_dataset[num].nunique()

hours-per-week     96
capital-gain      121
capital-loss       97
dtype: int64

In [42]:
adult_dataset[[target]].nunique()

income-per-year    2
dtype: int64

In [43]:
bins_cat = [7,16,6,14] #bins on the basis of unique values 
bins_num = [8,12,8]
target_bins = 2
protected_att_bins = [2,5,5,7]
version = 'mi_sklearn_04292022'

In [44]:
mi_xy_xa_hist_df = main(adult_dataset.copy(),cat,num,protected_attributes, target, bins_cat, bins_num , target_bins, protected_att_bins, version)


2it [00:00,  3.28it/s]
5it [00:01,  2.98it/s]
5it [00:01,  2.89it/s]
7it [00:02,  2.94it/s]
2it [00:00,  3.49it/s]
2it [00:01,  1.75it/s]
2it [00:01,  1.73it/s]
2it [00:01,  1.21it/s]


In [46]:
mi_xy_xa_hist = mi_xy_xa_hist_df[['A','I(X,A)','I(X,Y)']]
mi_xy_xa_hist = mi_xy_xa_hist.drop_duplicates(subset =["A"],keep='first')
mi_xy_xa_hist

,A,"I(X,A)","I(X,Y)"
0,sex,0.397488,0.290872
4,race,0.179103,0.290872
14,age_bins,0.413741,0.290872
24,marital-status,0.891120,0.290872


In [77]:
mi_xy_xa_hist['A'] = mi_xy_xa_hist['A'].str.replace('age_bins', 'age')

mi_xy_xa_hist.columns = ['A','I(X,A)_hist','I(X,Y)_hist']
mi_xy_xa_hist


,A,"I(X,A)_hist","I(X,Y)_hist"
0,sex,0.397488,0.290872
4,race,0.179103,0.290872
14,age,0.413741,0.290872
24,marital-status,0.891120,0.290872


### MI using RBIG

In [106]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder
from sklearn.utils import check_random_state
from mutual_info import MutualInfoRBIG
from sklearn.metrics import mutual_info_score

In [107]:
adult_dataset = pd.read_csv('adult_dataset.csv')
adult_dataset.drop('Unnamed: 0', axis = 1, inplace = True)

In [108]:
adult_dataset.columns

Index(['age', 'education-num', 'sex', 'capital-gain', 'capital-loss',
       'hours-per-week', 'income-per-year', 'education', 'marital-status',
       'native-country', 'occupation', 'race', 'relationship', 'workclass'],
      dtype='object')

In [109]:
#encoding the categorical columns, OrdinalEncoder - each unique category value is assigned an integer value
def categorical_feature_encoder(data,features):
    enc = {}
    
    for f in features:
        encoder = OrdinalEncoder()
        data[f] = encoder.fit_transform(data[[f]]).astype(int)
        enc[f] = encoder
    return data, enc

In [110]:
features = ['sex','education', 'marital-status','occupation', 'race', 'relationship', 'workclass']
features_encoded_data,enc = categorical_feature_encoder(adult_dataset,features)


In [111]:
#dropped the columns as the histogram approach did not consider both columns
features_encoded_data.drop('native-country', axis = 1, inplace = True)
features_encoded_data.drop('education-num', axis = 1, inplace = True)
features_encoded_data.head()


,age,sex,capital-gain,capital-loss,hours-per-week,income-per-year,education,marital-status,occupation,race,relationship,workclass
0,25.0,1,0.0,0.0,40.0,0.0,1,4,6,2,3,2
1,38.0,1,0.0,0.0,50.0,0.0,11,2,4,4,0,2
2,28.0,1,0.0,0.0,40.0,1.0,7,2,10,4,0,1
3,44.0,1,7688.0,0.0,40.0,1.0,15,2,6,2,0,2
4,34.0,1,0.0,0.0,30.0,0.0,0,4,7,4,1,2


In [112]:
X = ['capital-gain', 'capital-loss','hours-per-week', 'education','occupation', 
     'relationship', 'workclass']
Y = ['income-per-year']


In [113]:
def calculate_mi_RBIG(data,X,Y):

    # Initialize RBIG class
    rbig_model = MutualInfoRBIG(max_layers=1000, rotation= "PCA",zero_tolerance = 40)

    # fit model to the data
    rbig_model.fit(data[X], data[Y]);

    H_rbig = rbig_model.mutual_info()
    
    return H_rbig


In [114]:
rbig_mi_xy = calculate_mi_RBIG(features_encoded_data,X,Y)
rbig_mi_xy

Info Red: 0.00e+00: 100%|███████████████████| 1000/1000 [01:55<00:00,  8.63it/s]


4.470253284179309

In [115]:
X = ['hours-per-week', 'capital-gain', 'capital-loss','workclass','education', 'relationship', 'occupation'] 
A = ['sex','race','age', 'marital-status']


In [116]:
def calculate_miscore_xa(data,X,A):
    mis_xa = []
    p_attribute = []
    for a in A:
        rbig_model = MutualInfoRBIG(max_layers = 1000,rotation= "PCA",zero_tolerance = 40)
        rbig_model.fit(data[X], data[[a]]);
        mi_rbig = rbig_model.mutual_info()
        mis_xa.append(mi_rbig)
        p_attribute.append(a)  
    mis_xa_adult_dataFrame = pd.DataFrame({'A':p_attribute, 'I(Xi,A)': mis_xa})
    return mis_xa_adult_dataFrame


In [117]:
rbig_mi_xa = calculate_miscore_xa(features_encoded_data,X,A)
rbig_mi_xa

Info Red: 2.26e+00:   0%|                              | 0/1000 [00:00<?, ?it/s]/Users/surbhirathore/.local/lib/python3.7/site-packages/sklearn/decomposition/_pca.py:501: RuntimeWarning: invalid value encountered in true_divide
  explained_variance_ratio_ = explained_variance_ / total_var
Info Red: 0.00e+00:   0%|                              | 0/1000 [00:00<?, ?it/s]/Users/surbhirathore/.local/lib/python3.7/site-packages/sklearn/decomposition/_pca.py:501: RuntimeWarning: invalid value encountered in true_divide
  explained_variance_ratio_ = explained_variance_ / total_var
Info Red: 0.00e+00:   0%|                              | 0/1000 [00:00<?, ?it/s]/Users/surbhirathore/.local/lib/python3.7/site-packages/sklearn/decomposition/_pca.py:501: RuntimeWarning: invalid value encountered in true_divide
  explained_variance_ratio_ = explained_variance_ / total_var
Info Red: 0.00e+00:   0%|                              | 0/1000 [00:00<?, ?it/s]/Users/surbhirathore/.local/lib/python3.7/site-pac

Info Red: 0.00e+00:   2%|▍                    | 20/1000 [00:00<00:10, 93.83it/s]/Users/surbhirathore/.local/lib/python3.7/site-packages/sklearn/decomposition/_pca.py:501: RuntimeWarning: invalid value encountered in true_divide
  explained_variance_ratio_ = explained_variance_ / total_var
Info Red: 0.00e+00:   3%|▋                    | 30/1000 [00:00<00:10, 94.04it/s]/Users/surbhirathore/.local/lib/python3.7/site-packages/sklearn/decomposition/_pca.py:501: RuntimeWarning: invalid value encountered in true_divide
  explained_variance_ratio_ = explained_variance_ / total_var
Info Red: 0.00e+00:   3%|▋                    | 30/1000 [00:00<00:10, 94.04it/s]/Users/surbhirathore/.local/lib/python3.7/site-packages/sklearn/decomposition/_pca.py:501: RuntimeWarning: invalid value encountered in true_divide
  explained_variance_ratio_ = explained_variance_ / total_var
Info Red: 0.00e+00:   3%|▋                    | 30/1000 [00:00<00:10, 94.04it/s]/Users/surbhirathore/.local/lib/python3.7/site-pac

Info Red: 0.00e+00:   1%|▏                     | 9/1000 [00:00<00:11, 88.39it/s]/Users/surbhirathore/.local/lib/python3.7/site-packages/sklearn/decomposition/_pca.py:501: RuntimeWarning: invalid value encountered in true_divide
  explained_variance_ratio_ = explained_variance_ / total_var
Info Red: 0.00e+00:   1%|▏                     | 9/1000 [00:00<00:11, 88.39it/s]/Users/surbhirathore/.local/lib/python3.7/site-packages/sklearn/decomposition/_pca.py:501: RuntimeWarning: invalid value encountered in true_divide
  explained_variance_ratio_ = explained_variance_ / total_var
Info Red: 0.00e+00:   1%|▏                     | 9/1000 [00:00<00:11, 88.39it/s]/Users/surbhirathore/.local/lib/python3.7/site-packages/sklearn/decomposition/_pca.py:501: RuntimeWarning: invalid value encountered in true_divide
  explained_variance_ratio_ = explained_variance_ / total_var
Info Red: 0.00e+00:   2%|▍                    | 19/1000 [00:00<00:10, 94.89it/s]/Users/surbhirathore/.local/lib/python3.7/site-pac

Info Red: 0.00e+00:   0%|                              | 0/1000 [00:00<?, ?it/s]/Users/surbhirathore/.local/lib/python3.7/site-packages/sklearn/decomposition/_pca.py:501: RuntimeWarning: invalid value encountered in true_divide
  explained_variance_ratio_ = explained_variance_ / total_var
Info Red: 0.00e+00:   0%|                              | 0/1000 [00:00<?, ?it/s]/Users/surbhirathore/.local/lib/python3.7/site-packages/sklearn/decomposition/_pca.py:501: RuntimeWarning: invalid value encountered in true_divide
  explained_variance_ratio_ = explained_variance_ / total_var
Info Red: 0.00e+00:   0%|                              | 0/1000 [00:00<?, ?it/s]/Users/surbhirathore/.local/lib/python3.7/site-packages/sklearn/decomposition/_pca.py:501: RuntimeWarning: invalid value encountered in true_divide
  explained_variance_ratio_ = explained_variance_ / total_var
Info Red: 0.00e+00:   0%|                              | 0/1000 [00:00<?, ?it/s]/Users/surbhirathore/.local/lib/python3.7/site-pac

Info Red: 0.00e+00:   3%|▋                    | 30/1000 [00:00<00:10, 94.17it/s]/Users/surbhirathore/.local/lib/python3.7/site-packages/sklearn/decomposition/_pca.py:501: RuntimeWarning: invalid value encountered in true_divide
  explained_variance_ratio_ = explained_variance_ / total_var
Info Red: 0.00e+00:   3%|▋                    | 30/1000 [00:00<00:10, 94.17it/s]/Users/surbhirathore/.local/lib/python3.7/site-packages/sklearn/decomposition/_pca.py:501: RuntimeWarning: invalid value encountered in true_divide
  explained_variance_ratio_ = explained_variance_ / total_var
Info Red: 0.00e+00:   3%|▋                    | 30/1000 [00:00<00:10, 94.17it/s]/Users/surbhirathore/.local/lib/python3.7/site-packages/sklearn/decomposition/_pca.py:501: RuntimeWarning: invalid value encountered in true_divide
  explained_variance_ratio_ = explained_variance_ / total_var
Info Red: 0.00e+00:   3%|▋                    | 30/1000 [00:00<00:10, 94.17it/s]/Users/surbhirathore/.local/lib/python3.7/site-pac

,A,"I(Xi,A)"
0,sex,3.658593
1,race,3.658593
2,age,0.531108
3,marital-status,3.658593


In [118]:
rbig_mi_xa['I(Xi,Y)'] = 4.470253284179309
mi_xy_xa_rbig = rbig_mi_xa
mi_xy_xa_rbig.columns = ['A','I(X,A)_rbig','I(X,Y)_rbig']
mi_xy_xa_rbig

,A,"I(X,A)_rbig","I(X,Y)_rbig"
0,sex,3.658593,4.470253
1,race,3.658593,4.470253
2,age,0.531108,4.470253
3,marital-status,3.658593,4.470253


# Merged Table

In [119]:
def merged_mi(data1,data2):
    merged_mi = pd.merge(data1,data2, on=['A'], how = 'inner') 
    return merged_mi

In [120]:
merged_mi_hist_rbig = merged_mi(mi_xy_xa_hist,mi_xy_xa_rbig)
merged_mi_hist_rbig

,A,"I(X,A)_hist","I(X,Y)_hist","I(X,A)_rbig","I(X,Y)_rbig"
0,sex,0.397488,0.290872,3.658593,4.470253
1,race,0.179103,0.290872,3.658593,4.470253
2,age,0.413741,0.290872,0.531108,4.470253
3,marital-status,0.891120,0.290872,3.658593,4.470253


In [121]:
adult_dataset = pd.read_csv('adult_dataset.csv')
adult_dataset.drop('Unnamed: 0', axis = 1, inplace = True)

In [122]:
ad = adult_dataset[['sex','race','age','marital-status']].nunique().reset_index()

In [123]:
ad.columns = ['A','unique_vals']
ad

,A,unique_vals
0,sex,2
1,race,5
2,age,74
3,marital-status,7


In [124]:
merged_mi_hist_rbig = merged_mi(merged_mi_hist_rbig,ad)
merged_mi_hist_rbig

,A,"I(X,A)_hist","I(X,Y)_hist","I(X,A)_rbig","I(X,Y)_rbig",unique_vals
0,sex,0.397488,0.290872,3.658593,4.470253,2
1,race,0.179103,0.290872,3.658593,4.470253,5
2,age,0.413741,0.290872,0.531108,4.470253,74
3,marital-status,0.891120,0.290872,3.658593,4.470253,7
